# Notebook for Airconics examples 

This IPython notebook contains examples for generating and rendering the AirCONICS parametric transonic airliner example using the interactive `WebServer` from [PythonOCC-contrib](https://github.com/tpaviot/pythonocc-contrib).

In [ ]:
import airconics
import airconics.AirCONICStools as act

## Parameter Definitions

In [ ]:
Propulsion=1
EngineDia=2.9
FuselageScaling=[55.902, 55.902, 55.902]
NoseLengthRatio=0.182
TailLengthRatio=0.293
WingScaleFactor=44.56
WingChordFactor=1.0
Topology=1
EngineSpanStation=0.31
EngineCtrBelowLE=0.3558
EngineCtrFwdOfLE=0.9837
Scarf_deg=3

# Derived Parameters
FuselageHeight = FuselageScaling[2]*0.105
FuselageLength = FuselageScaling[0]
FuselageWidth  = FuselageScaling[1]*0.106
WingApex = [0.1748*FuselageLength,0,-0.0523*FuselageHeight]
# Fin:
FinChordFact = 1.01
FinScaleFact = WingScaleFactor/2.032
# TailPlane
TPChordFact = 1.01
TPScaleFact = WingScaleFactor * 0.388
# Engine:
NacelleLength = 1.95*EngineDia


## Wing, Transonic Airliner

TODO: I tried making the lifting surface loft through airfoil sections as a solid loft, however the output stp file was not correct in Rhino (may be because TE and ends are not closed)

In [ ]:
# Import all example functional definitions for the Common Research Model (CRM) Wing:
from airconics.examples.wing_example_transonic_airliner import *

# Position of the apex of the wing
P = WingApex

# Class definition
NSeg = 11
ChordFactor = 1
ScaleFactor = 50

# Generate (surface building is done during construction of the class)
Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
                                        myDihedralFunctionAirliner, 
                                        myTwistFunctionAirliner, 
                                        myChordFunctionAirliner, 
                                        myAirfoilFunctionAirliner,
                                        SegmentNo=NSeg,
                                        ScaleFactor=WingScaleFactor,
                                        ChordFactor=WingChordFactor)

# Evaluate the root chord:
RootChord = Wing.RootChord


## Tailplane, Transonic Airliner

In [ ]:
from OCC.gp import gp_Ax1, gp_Pnt, gp_Dir
from airconics.examples.tailplane_example_transonic_airliner import *

# Position of the apex of the fin
P = [36.98-0.49-0.02, 0.0, 2.395-0.141]

SegmentNo = 10

Fin = liftingsurface.LiftingSurface(P, mySweepAngleFunctionFin,
                                    myDihedralFunctionFin,
                                    myTwistFunctionFin,
                                    myChordFunctionFin,
                                    myAirfoilFunctionFin,
                                    SegmentNo=SegmentNo,
                                    ChordFactor=FinChordFact,
                                    ScaleFactor=FinScaleFact)

#    Create the rotation axis centered at the apex point in the x direction
RotAxis = gp_Ax1(gp_Pnt(*P), gp_Dir(1, 0, 0))


Fin.RotateComponents(RotAxis, 90)

# Position of the apex of the tailplane
P = [43, 0.000, 1.633+0.02]

SegmentNo = 100
ChordFactor = 1.01
ScaleFactor = 17.3

TP = liftingsurface.LiftingSurface(P, mySweepAngleFunctionTP,
                                   myDihedralFunctionTP,
                                   myTwistFunctionTP,
                                   myChordFunctionTP,
                                   myAirfoilFunctionTP,
                                   SegmentNo=SegmentNo,
                                   ChordFactor=TPChordFact,
                                   ScaleFactor=TPScaleFact)


## Fuselage Transonic Airliner

In [ ]:
from airconics.fuselage_oml import Fuselage

Fus = Fuselage(NoseLengthRatio, TailLengthRatio, Scaling=FuselageScaling,
             NoseCoordinates=[0., 0., 0],
             CylindricalMidSection=True,
             Maxi_attempt=5)

In [ ]:
# Export (can be commented out)
# act.export_STEPFile([Fus['OML']], 'fuselage.stp')

## Wing-Body Fairing:

In [ ]:
# WingBodyFairing:
from airconics.base import AirconicsShape
WTBFZ = RootChord*0.009 #787: 0.2
WTBFheight = 1.8*0.1212*RootChord #787:2.7
WTBFwidth = 1.08*FuselageWidth
WTBFXCentre = WingApex[0] + RootChord/2.0 + RootChord*0.1297 # 787: 23.8
WTBFlength = 1.167*RootChord #787:26

WBF_shape = act.make_ellipsoid([WTBFXCentre, 0, WTBFZ], WTBFlength, WTBFwidth, WTBFheight)
WBF = AirconicsShape(components={'WBF': WBF_shape})


## Engine + Pylon

In [ ]:
from airconics import engine

EngineSection, Chord = act.CutSect(Wing['Surface'], EngineSpanStation)
CEP = Chord.EndPoint()
Centreloc = [CEP.X()-EngineCtrFwdOfLE*NacelleLength,
            CEP.Y(), 
            CEP.Z()-EngineCtrBelowLE*NacelleLength]

eng =  engine.Engine(Chord,
                     CentreLocation=Centreloc,
                     ScarfAngle=Scarf_deg,
                     HighlightRadius=EngineDia/2.0,
                     MeanNacelleLength=NacelleLength)

## Miscelaneous operations

In [ ]:
# Trim the inboard section of the main wing:
CutCirc = act.make_circle3pt([0,WTBFwidth/4.,-45], [0,WTBFwidth/4.,45], [90,WTBFwidth/4.,0])
CutCircDisk = act.PlanarSurf(CutCirc)
Wing['Surface'] = act.TrimShapebyPlane(Wing['Surface'], CutCircDisk)

#Mirror the main wing and tailplane:
Wing2 = Wing.MirrorComponents(plane='xz')
TP2 = TP.MirrorComponents(plane='xz')
eng2 = eng.MirrorComponents(plane='xz')

## Ipython Cell Renderer:

In [ ]:
# Note : This seems to make objects render in ipython notebook (only works for shapes?)

from airconics.Addons.WebServer import TornadoWeb
renderer = TornadoWeb.TornadoWebRenderer()
#    display all entities:
# Fuselage and wing-body fairing
Fus.Display(renderer)
WBF.Display(renderer)

# #The Wings:
Wing.Display(renderer)
Wing2.Display(renderer)

#The Tailplane:
TP.Display(renderer)
TP2.Display(renderer)

#The Fin:
Fin.Display(renderer)

#The Engines:
eng.Display(renderer)
eng2.Display(renderer)

# Finally show the renderer
renderer

## Development

In [ ]:
# # This cell can be used to start a freecad window, create a new document and transfer the airconics
# # Wing shape to the viewer
# # NOTE:requires the same version of Open CASCADE though and is buggy, users should avoid this for now

# import sys
# sys.path.append('/usr/lib/freecad/lib')
# import FreeCAD
# import FreeCADGui
# %gui qt
# FreeCADGui.showMainWindow()

# doc = FreeCAD.newDocument()

# import Part

# import airconics 
# # Import all example functional definitions for the Common Research Model (CRM) Wing:
# from airconics.examples.wing_example_transonic_airliner import *

# # Position of the apex of the wing
# P = [0., 0., 0.]

# # Class definition
# NSeg = 10
# ChordFactor = 1
# ScaleFactor = 50

# Wing = liftingsurface.LiftingSurface(P, mySweepAngleFunctionAirliner, 
#     myDihedralFunctionAirliner, 
#     myTwistFunctionAirliner, 
#     myChordFunctionAirliner, 
#     myAirfoilFunctionAirliner, SegmentNo=NSeg, ScaleFactor=ScaleFactor)
# surf = Wing['Surface']
# w = Part.__fromPythonOCC__(surf)
# doc.recompute()
# Part.show(w)